## Predicting Median Income in Each SA2 Region Using Historical Data and Regression

In this notebook, we use historical income data to predict future median income in each SA2 region. By applying a regression model to past income trends, we aim to forecast future income levels based on historical patterns.




In [2]:
import pandas as pd

In [3]:
df = pd.read_excel("/root/project-2-group-real-estate-industry-project-34/data/landing/Table 1 - Total income, earners and summary statistics by geography, 2016-17 to 2020-21.xlsx",sheet_name='Table 1.4',skiprows = 5)

In [6]:
# After observing, we will likely select specific unnamed columns that correspond to the relevant information
df_filtered = df[['Unnamed: 0', 'Unnamed: 1', 'Median ($)', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21']]

# Rename the columns for clarity
df_filtered.columns = ['SA2', 'SA2 Name', 'Median ($) 2016-17', 'Median ($) 2017-18', 'Median ($) 2018-19', 'Median ($) 2019-20', 'Median ($) 2020-21']

# Drop the first row (which is likely the header or unnecessary data)
df_filtered = df_filtered.drop(index=0)

# Display the filtered DataFrame
df_filtered.head()

,SA2,SA2 Name,Median ($) 2016-17,Median ($) 2017-18,Median ($) 2018-19,Median ($) 2019-20,Median ($) 2020-21
1,Australia,NaN,"48,083","49,805","51,389","52,338","54,890"
2,New South Wales,NaN,"48,394","50,153","51,818","52,849","55,854"
3,101021007,Braidwood,"40,790","42,003","41,593","44,246","46,640"
4,101021008,Karabar,"57,460","59,295","61,777","62,946","65,564"
5,101021009,Queanbeyan,"55,033","57,848","60,119","61,724","63,528"


In [7]:
# FIlter SA2s in Victoria
df_filtered = df_filtered[df_filtered['SA2'].astype(str).str.startswith('2')]


In [8]:
df_filtered.head()

,SA2,SA2 Name,Median ($) 2016-17,Median ($) 2017-18,Median ($) 2018-19,Median ($) 2019-20,Median ($) 2020-21
646,201011001,Alfredton,"50,596","52,448","53,932","55,204","58,036"
647,201011002,Ballarat,"50,093","51,736","53,688","53,784","55,998"
648,201011005,Buninyong,"48,877","51,034","52,377","54,308","56,408"
649,201011006,Delacombe,"46,176","47,759","49,909","51,915","52,792"
650,201011007,Smythes Creek,"48,250","49,822","53,154","53,981","57,516"


In [9]:
# Rename the columns manually by replacing the year range with the final year
df_filtered.columns = ['SA2', 'SA2 Name', '2017', '2018', '2019', '2020', '2021']

# Display the renamed columns
df_filtered.head()

,SA2,SA2 Name,2017,2018,2019,2020,2021
646,201011001,Alfredton,"50,596","52,448","53,932","55,204","58,036"
647,201011002,Ballarat,"50,093","51,736","53,688","53,784","55,998"
648,201011005,Buninyong,"48,877","51,034","52,377","54,308","56,408"
649,201011006,Delacombe,"46,176","47,759","49,909","51,915","52,792"
650,201011007,Smythes Creek,"48,250","49,822","53,154","53,981","57,516"


In [10]:
# Find rows where 'np' appears in the DataFrame
np_rows = df_filtered[df_filtered.isin(['np']).any(axis=1)]

# Display the rows containing 'np'
print(np_rows)

           SA2                        SA2 Name    2017    2018    2019  \
734  205021083                       Lake King      np      np      np   
785  206041507  Royal Botanic Gardens Victoria  41,784      np  17,122   
907  210011227                Essendon Airport  24,967  27,711  22,966   

       2020    2021  
734      np      np  
785      np  23,341  
907  39,293      np  


In this code, we perform several preprocessing steps on the SA2 dataset to clean and prepare it for further analysis. First, we define the relevant columns (`2017`, `2018`, `2019`, `2020`, `2021`) and remove commas to ensure proper numeric conversion. We remove the row with 'Lake King' in the 'SA2 Name' column, as it's values are all np, not allowing for analysis. To handle missing data, we replace occurrences of 'np' with `NaN`. After that, we convert the specified columns to numeric types using `pd.to_numeric`, coercing any remaining invalid entries to `NaN`. Finally, we apply linear interpolation across the relevant columns to estimate and fill missing values based on the trends in the data. 


In [11]:
# Remove commas and convert columns to numeric (float) for the relevant columns
columns_to_convert = ['2017', '2018', '2019', '2020', '2021']

# Remove the row with 'Lake King' in the 'SA2 Name' column
df_filtered = df_filtered[df_filtered['SA2 Name'] != 'Lake King']

# Replace 'np' with NaN to handle missing data
df_filtered.replace('np', pd.NA, inplace=True)

# Convert relevant columns to numeric, coercing errors (turning non-numeric into NaN)
columns_to_convert = ['2017', '2018', '2019', '2020', '2021']
df_filtered[columns_to_convert] = df_filtered[columns_to_convert].replace({',': ''}, regex=True)
df_filtered[columns_to_convert] = df_filtered[columns_to_convert].apply(pd.to_numeric, errors='coerce')

# Use linear interpolation to predict missing 'np' values
df_filtered[columns_to_convert] = df_filtered[columns_to_convert].interpolate(method='linear', axis=1)

# Display the rows where 'np' values were replaced (e.g., rows 785 and 907)
np_replaced_rows = df_filtered.loc[[785, 907], columns_to_convert]
print(np_replaced_rows)


        2017     2018     2019     2020     2021
785  41784.0  29453.0  17122.0  20231.5  23341.0
907  24967.0  27711.0  22966.0  39293.0  39293.0


In this code, we predict future median income for each SA2 region using a Linear Regression model trained on historical data from 2017 to 2021. We first define the years we want to predict, including both past (2011 to 2016) and future years (2022 to 2027). For each SA2 region (represented by each row in the dataset), we prepare the training data using the years 2017 to 2021 as features and the corresponding income values as targets. We then initialize and fit a Linear Regression model to this data. After training, we predict the income for the specified years, storing the predicted values in a dictionary. Finally, we add the predictions to the DataFrame as new columns for each year from 2011 to 2027, and display the updated DataFrame to verify that the predictions have been successfully added.


In [12]:
from sklearn.linear_model import LinearRegression
import numpy as np

# Initialize a dictionary to store predictions for each year
years_to_predict = [2011, 2012, 2013, 2014, 2015, 2016, 2022, 2023, 2024, 2025, 2026, 2027]
predictions = {year: [] for year in years_to_predict}

# Loop over each SA2 (each row) and fit a linear regression model using 2017-2021 data
for index, row in df_filtered.iterrows():
    # Prepare the training data for this specific SA2 (2017–2021)
    X_train = np.array([[2017], [2018], [2019], [2020], [2021]])  # The years as features
    y_train = row[['2017', '2018', '2019', '2020', '2021']].values  # Income values as targets
    
    # Initialize the Linear Regression model
    model = LinearRegression()
    
    # Fit the model on the training data for this SA2
    model.fit(X_train, y_train)
    
    # Predict the income for the years 2022 to 2027
    X_predict = np.array([[2011], [2012], [2013], [2014], [2015], [2016], [2022], [2023], [2024], [2025], [2026], [2027]])  # Future years
    predicted_values = model.predict(X_predict)
    
    # Store the predictions for this SA2
    for i, year in enumerate(years_to_predict):
        predictions[year].append(predicted_values[i])

# Add the predictions for each year to the DataFrame
for year in years_to_predict:
    df_filtered[str(year)] = predictions[year]

# Display the updated DataFrame with predictions for 2022 to 2027
df_filtered.head() 

,SA2,SA2 Name,2017,2018,2019,2020,2021,2011,2012,2013,2014,2015,2016,2022,2023,2024,2025,2026,2027
646,201011001,Alfredton,50596.0,52448.0,53932.0,55204.0,58036.0,39934.4,41698.0,43461.6,45225.2,46988.8,48752.4,59334.0,61097.6,62861.2,64624.8,66388.4,68152.0
647,201011002,Ballarat,50093.0,51736.0,53688.0,53784.0,55998.0,41973.4,43359.2,44745.0,46130.8,47516.6,48902.4,57217.2,58603.0,59988.8,61374.6,62760.4,64146.2
648,201011005,Buninyong,48877.0,51034.0,52377.0,54308.0,56408.0,37932.0,39765.6,41599.2,43432.8,45266.4,47100.0,58101.6,59935.2,61768.8,63602.4,65436.0,67269.6
649,201011006,Delacombe,46176.0,47759.0,49909.0,51915.0,52792.0,35799.8,37538.6,39277.4,41016.2,42755.0,44493.8,54926.6,56665.4,58404.2,60143.0,61881.8,63620.6
650,201011007,Smythes Creek,48250.0,49822.0,53154.0,53981.0,57516.0,34391.8,36660.9,38930.0,41199.1,43468.2,45737.3,59351.9,61621.0,63890.1,66159.2,68428.3,70697.4


In [13]:
# Copy the DataFrame
df_filtered_sorted = df_filtered.copy()

# Get the columns that are years and sort them by year
sorted_columns = sorted([col for col in df_filtered.columns if col.isdigit()], key=lambda x: int(x))

# Rename the columns to "20xx_median_income"
renamed_columns = {col: f"{col}_median_income" for col in sorted_columns}

# Rename the columns in the DataFrame
df_filtered_sorted = df_filtered_sorted.rename(columns=renamed_columns)

# Reorder the DataFrame to have 'SA2', 'SA2 Name' followed by the sorted year columns
df_filtered_sorted = df_filtered_sorted[['SA2', 'SA2 Name'] + list(renamed_columns.values())]

# Display the updated DataFrame
df_filtered_sorted.head()


,SA2,SA2 Name,2011_median_income,2012_median_income,2013_median_income,2014_median_income,2015_median_income,2016_median_income,2017_median_income,2018_median_income,2019_median_income,2020_median_income,2021_median_income,2022_median_income,2023_median_income,2024_median_income,2025_median_income,2026_median_income,2027_median_income
646,201011001,Alfredton,39934.4,41698.0,43461.6,45225.2,46988.8,48752.4,50596.0,52448.0,53932.0,55204.0,58036.0,59334.0,61097.6,62861.2,64624.8,66388.4,68152.0
647,201011002,Ballarat,41973.4,43359.2,44745.0,46130.8,47516.6,48902.4,50093.0,51736.0,53688.0,53784.0,55998.0,57217.2,58603.0,59988.8,61374.6,62760.4,64146.2
648,201011005,Buninyong,37932.0,39765.6,41599.2,43432.8,45266.4,47100.0,48877.0,51034.0,52377.0,54308.0,56408.0,58101.6,59935.2,61768.8,63602.4,65436.0,67269.6
649,201011006,Delacombe,35799.8,37538.6,39277.4,41016.2,42755.0,44493.8,46176.0,47759.0,49909.0,51915.0,52792.0,54926.6,56665.4,58404.2,60143.0,61881.8,63620.6
650,201011007,Smythes Creek,34391.8,36660.9,38930.0,41199.1,43468.2,45737.3,48250.0,49822.0,53154.0,53981.0,57516.0,59351.9,61621.0,63890.1,66159.2,68428.3,70697.4


In [15]:
df_filtered_sorted.to_parquet("income_by_SA2.parquet")

In [16]:
rent_df = pd.read_parquet("/root/project-2-group-real-estate-industry-project-34/data/landing/preprocessed_rent_data.parquet")

In [17]:
rent_df.head()

,address,state,suburb,bedrooms,bathrooms,propertyTypes,carspaces,date_listed,latitude,longitude,is_new_development,price,propertyId,is_furnished,year,month,day,SA2_CODE21
0,"1201/55 Queens Road, Melbourne VIC 3000",vic,Melbourne,2.0,2.0,[apartmentUnitFlat],2.0,2009-05-08,-37.847967,144.977640,False,1000.0,YH-6838-CG,True,2009,5,8,206051128
1,"211/G04K Powlett Street, East Melbourne VIC 3002",vic,East Melbourne,3.0,2.0,[apartmentUnitFlat],1.0,2009-05-08,-37.812575,144.985854,False,680.0,None,True,2009,5,8,206041119
2,Rye VIC 3941,vic,Rye,3.0,2.0,[house],4.0,2009-05-08,-38.373284,144.817655,False,1200.0,None,False,2009,5,8,214021383
3,"1204/454 St Kilda Road, Melbourne St Kilda Roa...",vic,Melbourne St Kilda Road,3.0,3.0,[apartmentUnitFlat],2.0,2009-05-11,-37.839405,144.976224,False,1300.0,None,False,2009,5,11,206051128
4,"211 Wellington Pde Sth, East Melbourne VIC 3002",vic,East Melbourne,3.0,3.0,[apartmentUnitFlat],2.0,2009-05-11,-37.816637,144.977522,False,720.0,None,False,2009,5,11,206041119


In [18]:
# Step 1: Reshape the income dataset by melting it to have 'year' as a separate column
df_melted_income = pd.melt(df_filtered_sorted, 
                           id_vars=['SA2', 'SA2 Name'], 
                           var_name='year', 
                           value_name='Income (Median)')

# Extract the year from the column names like '2022_median_income' and convert it to int
df_melted_income['year'] = df_melted_income['year'].str.extract('(\d{4})').astype(int)

# Step 2: Merge the rent_df with the melted income data on 'SA2_CODE21' and 'year'
rent_df = rent_df.merge(df_melted_income, how='left', left_on=['SA2_CODE21', 'year'], right_on=['SA2', 'year'])

# Step 3: Drop the extra 'SA2' column from the merged data
rent_df.drop(columns=['SA2'], inplace=True)

# Display the updated DataFrame
rent_df.head()


,address,state,suburb,bedrooms,bathrooms,propertyTypes,carspaces,date_listed,latitude,longitude,is_new_development,price,propertyId,is_furnished,year,month,day,SA2_CODE21,SA2 Name,Income (Median)
0,"1201/55 Queens Road, Melbourne VIC 3000",vic,Melbourne,2.0,2.0,[apartmentUnitFlat],2.0,2009-05-08,-37.847967,144.977640,False,1000.0,YH-6838-CG,True,2009,5,8,206051128,NaN,NaN
1,"211/G04K Powlett Street, East Melbourne VIC 3002",vic,East Melbourne,3.0,2.0,[apartmentUnitFlat],1.0,2009-05-08,-37.812575,144.985854,False,680.0,None,True,2009,5,8,206041119,NaN,NaN
2,Rye VIC 3941,vic,Rye,3.0,2.0,[house],4.0,2009-05-08,-38.373284,144.817655,False,1200.0,None,False,2009,5,8,214021383,NaN,NaN
3,"1204/454 St Kilda Road, Melbourne St Kilda Roa...",vic,Melbourne St Kilda Road,3.0,3.0,[apartmentUnitFlat],2.0,2009-05-11,-37.839405,144.976224,False,1300.0,None,False,2009,5,11,206051128,NaN,NaN
4,"211 Wellington Pde Sth, East Melbourne VIC 3002",vic,East Melbourne,3.0,3.0,[apartmentUnitFlat],2.0,2009-05-11,-37.816637,144.977522,False,720.0,None,False,2009,5,11,206041119,NaN,NaN


In [19]:
model_df = pd.read_csv("/root/project-2-group-real-estate-industry-project-34/data/landing/test_data.csv")

In [21]:
# Ensure both SA2_CODE21 and SA2 columns are strings
model_df['SA2_CODE21'] = model_df['SA2_CODE21'].astype(str)
df_melted_income['SA2'] = df_melted_income['SA2'].astype(str)

# Step 2: Merge the rent_df with the melted income data on 'SA2_CODE21' and 'year'
model_df = model_df.merge(df_melted_income, how='left', left_on=['SA2_CODE21', 'year'], right_on=['SA2', 'year'])

# Step 3: Drop the extra 'SA2' column from the merged data
model_df.drop(columns=['SA2'], inplace=True)

# Display the updated DataFrame
model_df.head()


# Display the updated DataFrame
model_df.head()

,SA2_CODE21,bedrooms,bathrooms,year,distance_to_bus_stop,distance_to_train_stop,distance_to_tram_stop,distance_to_hospital,distance_to_mall,distance_to_park,...,distance_to_school,distance_to_uni,population,date_numeric,bed_bath_interaction,carspaces,is_furnished,crime,SA2 Name,Income (Median)
0,201011006,1,1,2025,0.741864,0.91511,1.091632,0.036647,0.007196,0.00017,...,0.03149,1.178579,15294,5295,1,1,0,1210.912850,Delacombe,60143.0
1,201011006,1,1,2026,0.741864,0.91511,1.091632,0.036647,0.007196,0.00017,...,0.03149,1.178579,16366,5660,1,1,0,1290.087583,Delacombe,61881.8
2,201011006,1,1,2027,0.741864,0.91511,1.091632,0.036647,0.007196,0.00017,...,0.03149,1.178579,17341,6025,1,1,0,1366.340312,Delacombe,63620.6
3,201011006,1,1,2025,0.741864,0.91511,1.091632,0.036647,0.007196,0.00017,...,0.03149,1.178579,15294,5295,1,0,0,1210.912850,Delacombe,60143.0
4,201011006,1,1,2026,0.741864,0.91511,1.091632,0.036647,0.007196,0.00017,...,0.03149,1.178579,16366,5660,1,0,0,1290.087583,Delacombe,61881.8


In [22]:
model_df.to_parquet("test_data_with_income.parquet")